### RAG PIPELINE : DATA INGESTION TO VECTOR DB

In [112]:

import os
from langchain_community.document_loaders import (
    DirectoryLoader,
    PyMuPDFLoader,
    PyPDFLoader,
)
from langchain_text_splitters import RecursiveCharacterTextSplitter
from pathlib import Path

# initialize to avoid unused import warning
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

In [113]:
#reading all the pdf files in the data/pdf directory
def process_all_pdfs(pdf_directory):
    all_documents = []
    pdf_dir = Path(pdf_directory)
    # print(type(pdf_dir)) #=> ye path object me convert krne ke liye hai :)
    
    pdf_files = list(pdf_dir.glob("**/*.pdf")) #all pdf files in the directory and subdirectories
    for pdf_file in pdf_files:
        print(f"Processing {pdf_file}...")
        try:
            loader = PyMuPDFLoader(str(pdf_file))
            documents = loader.load()
            # Add source information to metadata
            for doc in documents:
                doc.metadata["source_file"] = pdf_file.name
                doc.metadata["file_type"] = "pdf"
            all_documents.extend(documents)
        except Exception as e:
            print(f"Error processing {pdf_file}: {e}")
        finally:
            print(f"Finished processing {pdf_file}.")

    return all_documents

    

In [114]:
all_pdf_documents = process_all_pdfs("../data/pdf")

Processing ../data/pdf/Python Programming.pdf...
Finished processing ../data/pdf/Python Programming.pdf.
Processing ../data/pdf/LangChain.pdf...
Finished processing ../data/pdf/LangChain.pdf.


In [115]:
all_pdf_documents

[Document(metadata={'producer': 'pdfTeX-1.40.18', 'creator': 'TeX', 'creationdate': '2020-08-12T08:23:16+00:00', 'source': '../data/pdf/Python Programming.pdf', 'file_path': '../data/pdf/Python Programming.pdf', 'total_pages': 140, 'format': 'PDF 1.5', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2020-08-12T08:23:16+00:00', 'trapped': '', 'modDate': 'D:20200812082316Z', 'creationDate': 'D:20200812082316Z', 'page': 0, 'source_file': 'Python Programming.pdf', 'file_type': 'pdf'}, page_content='Python \nProgramming\nHans-Petter Halvorsen\nhttps://www.halvorsen.blog'),
 Document(metadata={'producer': 'pdfTeX-1.40.18', 'creator': 'TeX', 'creationdate': '2020-08-12T08:23:16+00:00', 'source': '../data/pdf/Python Programming.pdf', 'file_path': '../data/pdf/Python Programming.pdf', 'total_pages': 140, 'format': 'PDF 1.5', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2020-08-12T08:23:16+00:00', 'trapped': '', 'modDate': 'D:20200812082316Z', 'crea

In [116]:
def split_documents(documents, chunk_size=1000, chunk_overlap=200):
    """Split documents into smaller chunks for better RAG performance"""
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        separators=["\n\n", "\n", " ", ""],
    )
    split_docs = text_splitter.split_documents(documents)
    print(f"Splitted {len(documents)} documents into {len(split_docs)} chunks.")
    return split_docs

In [52]:
chunks = split_documents(documents=all_pdf_documents)

Splitted 154 documents into 256 chunks.


In [45]:
import numpy as np
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import uuid
from typing import List, Dict, Any, Tuple
from sklearn.metrics.pairwise import cosine_similarity


/Users/shauryagupta/Desktop/Dailoqa/RAG LEARNING/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [54]:
class EmbeddingsManager:
    '''Handles Document Embedding Generation using sentence-transformers and storage in ChromaDB'''
    def __init__(self, model_name: str = "all-MiniLM-L6-v2"):
        self.model_name = model_name
        self.model = None
        self._load_model()
    
    def _load_model(self):
        """Load the sentence transformer model for embedding generation"""
        try:
            print(f"Loading embedding model: {self.model_name}")
            self.model = SentenceTransformer(self.model_name)
            print(
                f"Model loaded successfully. Embedding dimension: {self.model.get_sentence_embedding_dimension()}"
            )

        except Exception as e:
            print(f"Error loading model {self.model_name}: {e}")
            raise e

    def generate_embeddings(self, texts: List[str]) -> np.ndarray:
        """Generate embeddings for a list of texts"""
        try:
            embeddings = self.model.encode(texts, convert_to_numpy=True, show_progress_bar=True)
            print(f"Generated embeddings shape: {embeddings.shape}")
            return embeddings
        except Exception as e:
            print(f"Error generating embeddings: {e}")
            raise e

embedding_manager = EmbeddingsManager()
embedding_manager

Loading embedding model: all-MiniLM-L6-v2


Loading weights: 100%|██████████| 103/103 [00:00<00:00, 2014.24it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Model loaded successfully. Embedding dimension: 384


In [71]:
# Vector Store Creation and Querying
import uuid
import numpy as np
import chromadb
from typing import List, Any

class VectorStore:
    """Manages document embeddings in a ChromaDB vector store"""

    def __init__(self, collection_name: str = "pdf_documents", persist_directory: str = "../data/vector_store"):
        """Initialize the vector store."""
        self.collection_name = collection_name
        self.persist_directory = persist_directory
        self.client = None
        self.collection = None
        self._initialize_store()

    def _initialize_store(self):
        """Initialize ChromaDB client and collection."""
        try:
            print(f"Initializing ChromaDB client with persist directory: {self.persist_directory}")
            # In newer versions of ChromaDB, use PersistentClient for persistent storage
            self.client = chromadb.PersistentClient(path=self.persist_directory)
            
            self.collection = self.client.get_or_create_collection(
                name=self.collection_name,
                metadata={"description": "Collection for PDF document embeddings"}
            )
            print(f"Vector store initialized. Collection: {self.collection_name}")
            print(f"Existing documents in collection: {self.collection.count()}")
        except Exception as e:
            print(f"Error initializing vector store: {e}")
            raise e

    def add_documents(self, documents: List[Any], embeddings: np.ndarray):
        """Add documents and embeddings to the vector store."""
        if len(documents) != len(embeddings):
            raise ValueError("Number of documents must match number of embeddings")
        print(f"Adding {len(documents)} documents to vector store...")
        ids = []
        metadatas = []
        documents_text = []
        embeddings_list = []
        for i, (doc, embedding) in enumerate(zip(documents, embeddings)):
            doc_id = f"doc_{uuid.uuid4().hex[:8]}_{i}"
            ids.append(doc_id)
            metadata = dict(doc.metadata)
            metadata["doc_index"] = i
            metadata["content_length"] = len(doc.page_content)
            metadatas.append(metadata)
            documents_text.append(doc.page_content)
            embeddings_list.append(embedding.tolist())
        try:
            self.collection.add(ids=ids, embeddings=embeddings_list, metadatas=metadatas, documents=documents_text)
            print(f"Successfully added {len(documents)} documents to vector store")
            print(f"Total documents in collection: {self.collection.count()}")
        except Exception as e:
            print(f"Error adding documents to vector store: {e}")
            raise e

vectorstore = VectorStore()
vectorstore

Initializing ChromaDB client with persist directory: ../data/vector_store
Vector store initialized. Collection: pdf_documents
Existing documents in collection: 256


In [ ]:
chunks

In [73]:
texts = [doc.page_content for doc in chunks]
# texts[:3]

#generating texts into embeddings
embeddings = embedding_manager.generate_embeddings(texts)

#adding into chromadb
vectorstore.add_documents(documents=chunks, embeddings=embeddings)

Batches: 100%|██████████| 8/8 [00:09<00:00,  1.15s/it]


Generated embeddings shape: (256, 384)
Adding 256 documents to vector store...
Successfully added 256 documents to vector store
Total documents in collection: 512


### rag retriver pipeline from datastore


In [ ]:
class RAGRetriever:
    """Handles query-based retrieval from the vector store"""
    
    def __init__(self, vector_store: VectorStore, embedding_manager: EmbeddingsManager):
        """
        Initialize the retriever
        
        Args:
            vector_store: Vector store containing document embeddings
            embedding_manager: Manager for generating query embeddings
        """
        self.vector_store = vector_store
        self.embedding_manager = embedding_manager
        
    def retrieve(
        self, query: str, top_k: int = 5, score_threshold: float = 0.0
    ) -> List[Dict[str, Any]]:
        """
        Retrieve relevant documents for a query

        Args:
            query: The search query
            top_k: Number of top results to return
            score_threshold: Minimum similarity score threshold

        Returns:
            List of dictionaries containing retrieved documents and metadata
        """
        print(f"Retrieving documents for query: '{query}'")
        print(f"Top K: {top_k}, Score threshold: {score_threshold}")

        # Generate query embedding
        query_embedding = self.embedding_manager.generate_embeddings([query])[0] #this will return a 2D array with shape (1, embedding_dim), we take the first element to get the 1D embedding vector   

        # Search in vector store
        try:
            results = self.vector_store.collection.query(
                query_embeddings=[query_embedding.tolist()], n_results=top_k
            )

            # Process results
            retrieved_docs = []

            if results["documents"] and results["documents"][0]:
                documents = results["documents"][0]
                metadatas = results["metadatas"][0]
                distances = results["distances"][0]
                ids = results["ids"][0]

                for i, (doc_id, document, metadata, distance) in enumerate(
                    zip(ids, documents, metadatas, distances)
                ):
                    # Convert distance to similarity score (ChromaDB uses cosine distance)
                    similarity_score = 1 - distance

                    if similarity_score >= score_threshold:
                        retrieved_docs.append(
                            {
                                "id": doc_id,
                                "content": document,
                                "metadata": metadata,
                                "similarity_score": similarity_score,
                                "distance": distance,
                                "rank": i + 1,
                            }
                        )

                print(f"Retrieved {len(retrieved_docs)} documents (after filtering)")
            else:
                print("No documents found")

            return retrieved_docs

        except Exception as e:
            print(f"Error during retrieval: {e}")
            return []


rag_retriever = RAGRetriever(vectorstore, embedding_manager)
        

In [76]:
rag_retriever


In [78]:
rag_retriever.retrieve("What is python?")


Retrieving documents for query: 'What is python?'
Top K: 5, Score threshold: 0.0


Batches: 100%|██████████| 1/1 [00:00<00:00,  1.03it/s]

Generated embeddings shape: (1, 384)
Retrieved 5 documents (after filtering)


[{'id': 'doc_39ccfc03_1',
  'content': 'Python Programming',
  'metadata': {'creationdate': '2020-08-12T08:23:16+00:00',
   'file_type': 'pdf',
   'keywords': '',
   'format': 'PDF 1.5',
   'total_pages': 140,
   'source_file': 'Python Programming.pdf',
   'title': '',
   'doc_index': 1,
   'page': 1,
   'creator': 'TeX',
   'producer': 'pdfTeX-1.40.18',
   'moddate': '2020-08-12T08:23:16+00:00',
   'source': '../data/pdf/Python Programming.pdf',
   'modDate': 'D:20200812082316Z',
   'content_length': 18,
   'file_path': '../data/pdf/Python Programming.pdf',
   'author': '',
   'subject': '',
   'creationDate': 'D:20200812082316Z',
   'trapped': ''},
  'similarity_score': 0.5663045048713684,
  'distance': 0.4336954951286316,
  'rank': 1},
 {'id': 'doc_e7d8cb3d_206',
  'content': 'Python Programming',
  'metadata': {'creator': 'TeX',
   'creationDate': 'D:20200812082316Z',
   'moddate': '2020-08-12T08:23:16+00:00',
   'creationdate': '2020-08-12T08:23:16+00:00',
   'source': '../data/pd

In [87]:

# from dotenv import load_dotenv

# load_dotenv()

# print(os.getenv("AZURE_OPENAI_API_KEY"))


# from langchain_openai import AzureChatOpenAI
# from langchain_core.prompts import PromptTemplate
# from langchain_core.messages import HumanMessage, SystemMessage


# model = AzureChatOpenAI(
#     azure_deployment=os.getenv("AZURE_DEPLOYMENT_NAME"),  # In Azure, model_name is usually the deployment name
#     api_key=os.getenv("AZURE_OPENAI_API_KEY"),
#     azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
#     api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
# )



In [88]:
# result = model.invoke("hey can i ask you something?")
# print(result)

In [89]:
import os
from dotenv import load_dotenv

load_dotenv()

print(os.getenv("GROQ_API_KEY"))


[REDACTED_API_KEY]


In [91]:
from langchain_groq import ChatGroq
from langchain_core.prompts import PromptTemplate
from langchain_core.messages import HumanMessage, SystemMessage


In [105]:
class GroqLLM:
    def __init__(self, model_name: str = "llama-3.3-70b-versatile", api_key: str = None):
        """
        Initialize Groq LLM

        Args:
            model_name: Groq model name (e.g., llama-3.3-70b-versatile, llama3-70b-8192)
            api_key: Groq API key (or set GROQ_API_KEY environment variable)
        """
        self.model_name = model_name
        self.api_key = api_key or os.environ.get("GROQ_API_KEY")

        if not self.api_key:
            raise ValueError(
                "Groq API key is required. Set GROQ_API_KEY environment variable or pass api_key parameter."
            )

        self.llm = ChatGroq(
            groq_api_key=self.api_key,
            model_name=self.model_name,
            temperature=0.1,
            max_tokens=1024,
        )

        print(f"Initialized Groq LLM with model: {self.model_name}")

    def generate_response(self, query: str, context: str, max_length: int = 500) -> str:
        """
        Generate response using retrieved context

        Args:
            query: User question
            context: Retrieved document context
            max_length: Maximum response length

        Returns:
            Generated response string
        """

        # Create prompt template
        prompt_template = PromptTemplate(
            input_variables=["context", "question"],
            template="""You are a helpful AI assistant. Use the following context to answer the question accurately and concisely.
Context: {context}
Question: {question}
Answer: Provide a clear and informative answer based on the context above. If the context doesn't contain enough information to answer the question, say so.""",
        )

        # Format the prompt
        formatted_prompt = prompt_template.format(context=context, question=query)

        try:
            # Generate response
            messages = [HumanMessage(content=formatted_prompt)]
            response = self.llm.invoke(messages)
            return response.content

        except Exception as e:
            return f"Error generating response: {str(e)}"

    def generate_response_simple(self, query: str, context: str) -> str:
        """Simple response generation without complex prompting"""
        simple_prompt = f"Based on this context: {context}\\n\\nQuestion: {query}\\n\\nAnswer:"
        try:
            messages = [HumanMessage(content=simple_prompt)]
            response = self.llm.invoke(messages)
            return response.content
        except Exception as e:
            return f"Error: {str(e)}"

In [106]:
groq = GroqLLM()
response = groq.llm.invoke("hey can i ask you something?")
response

Initialized Groq LLM with model: llama-3.3-70b-versatile


AIMessage(content="I'd be happy to help with any questions you have. Go ahead and ask away!", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 42, 'total_tokens': 61, 'completion_time': 0.050681729, 'completion_tokens_details': None, 'prompt_time': 0.003974126, 'prompt_tokens_details': None, 'queue_time': 0.048000234, 'total_time': 0.054655855}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_68f543a7cc', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019c3e85-73e7-71e1-9faf-7dbc595181ca-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 42, 'output_tokens': 19, 'total_tokens': 61})

In [ ]:
# Initialize Groq LLM (you'll need to set GROQ_API_KEY environment variable)
try:
    groq_llm = GroqLLM(api_key=os.getenv("GROQ_API_KEY"))
    print("Groq LLM initialized successfully!")
except ValueError as e:
    print(f"Warning: {e}")
    print("Please set your GROQ_API_KEY environment variable to use the LLM.")
    groq_llm = None


In [101]:
### get the context from the retriever and pass it to the LLM

response = rag_retriever.retrieve("Who is the writter of this langchain pdf")
response


Retrieving documents for query: 'Who is the writter of this langchain pdf'
Top K: 5, Score threshold: 0.0


Batches: 100%|██████████| 1/1 [00:08<00:00,  8.97s/it]


Generated embeddings shape: (1, 384)
Retrieved 2 documents (after filtering)


[{'id': 'doc_dc3eaf22_252',
  'content': '14\nVasilios Mavroudis\nReferences\n1. Josh Achiam, Steven Adler, Sandhini Agarwal, Lama Ahmad, Ilge Akkaya, Flo-\nrencia Leoni Aleman, Diogo Almeida, Janko Altenschmidt, Sam Altman, Shyamal\nAnadkat, et al. GPT-4 Technical Report. arXiv preprint arXiv:2303.08774, 2023.\n2. Harrison Chase.\nLangChain, Oct 2022.\nAvailable at https://github.com/\nlangchain-ai/langchain.\n3. LangChain, Inc. LangChain Documentation: Integration Providers. LangChain,\nInc., San Francisco, CA, 2024. Available at https://python.langchain.com/docs/\nintegrations/providers/.\n4. LangChain, Inc.\nLangChain Documentation: Key Concepts.\nLangChain, Inc.,\nSan Francisco, CA, 2024.\nAvailable at https://python.langchain.com/docs/\nconcepts/.\n5. LangChain, Inc.\nLangChain Documentation: LangServe.\nLangChain, Inc.,\nSan Francisco, CA, 2024.\nAvailable at https://python.langchain.com/docs/\nlangserve/.\n6. LangChain, Inc. LangChain Documentation: Security Best Practices. Lan

### Integration Vectordb Context pipeline With LLM output

In [108]:
### Simple RAG pipeline with Groq LLM
from langchain_groq import ChatGroq
import os
from dotenv import load_dotenv

load_dotenv()

### Initialize the Groq LLM (set your GROQ_API_KEY in environment)
groq_api_key = os.getenv("GROQ_API_KEY")

llm = ChatGroq(
    groq_api_key=groq_api_key,
    model_name="llama-3.3-70b-versatile",
    temperature=0.1,
    max_tokens=1024,
)


## 2. Simple RAG function: retrieve context + generate response
def rag_simple(query, retriever, llm, top_k=3):
    ## retriever the context
    results = retriever.retrieve(query, top_k=top_k)
    context = "\n\n".join([doc["content"] for doc in results]) if results else ""
    if not context:
        return "No relevant context found to answer the question."

    ## generate the answwer using GROQ LLM
    prompt = f"""Use the following context to answer the question concisely.
        Context:
        {context}

        Question: {query}

        Answer:"""

    response = llm.invoke([prompt.format(context=context, query=query)])
    return response.content

In [109]:
answer = rag_simple("What is attention mechanism?", rag_retriever, llm)
print(answer)


Retrieving documents for query: 'What is attention mechanism?'
Top K: 3, Score threshold: 0.0


Batches: 100%|██████████| 1/1 [00:07<00:00,  7.70s/it]

Generated embeddings shape: (1, 384)
Retrieved 0 documents (after filtering)
No relevant context found to answer the question.


In [110]:
# --- Enhanced RAG Pipeline Features ---
def rag_advanced(query, retriever, llm, top_k=5, min_score=0.2, return_context=False):
    """
    RAG pipeline with extra features:
    - Returns answer, sources, confidence score, and optionally full context.
    """
    results = retriever.retrieve(query, top_k=top_k, score_threshold=min_score)
    if not results:
        return {
            "answer": "No relevant context found.",
            "sources": [],
            "confidence": 0.0,
            "context": "",
        }

    # Prepare context and sources
    context = "\n\n".join([doc["content"] for doc in results])
    sources = [
        {
            "source": doc["metadata"].get(
                "source_file", doc["metadata"].get("source", "unknown")
            ),
            "page": doc["metadata"].get("page", "unknown"),
            "score": doc["similarity_score"],
            "preview": doc["content"][:300] + "...",
        }
        for doc in results
    ]
    confidence = max([doc["similarity_score"] for doc in results])

    # Generate answer
    prompt = f"""Use the following context to answer the question concisely.\nContext:\n{context}\n\nQuestion: {query}\n\nAnswer:"""
    response = llm.invoke([prompt.format(context=context, query=query)])

    output = {"answer": response.content, "sources": sources, "confidence": confidence}
    if return_context:
        output["context"] = context
    return output


# Example usage:
result = rag_advanced(
    "Hard Negative Mining Technqiues",
    rag_retriever,
    llm,
    top_k=3,
    min_score=0.1,
    return_context=True,
)
print("Answer:", result["answer"])
print("Sources:", result["sources"])
print("Confidence:", result["confidence"])
print("Context Preview:", result["context"][:300])


Retrieving documents for query: 'Hard Negative Mining Technqiues'
Top K: 3, Score threshold: 0.1


Batches: 100%|██████████| 1/1 [00:00<00:00,  2.93it/s]

Generated embeddings shape: (1, 384)
Retrieved 0 documents (after filtering)
Answer: No relevant context found.
Sources: []
Confidence: 0.0
Context Preview: 


In [111]:
# --- Advanced RAG Pipeline: Streaming, Citations, History, Summarization ---
from typing import List, Dict, Any
import time


class AdvancedRAGPipeline:
    def __init__(self, retriever, llm):
        self.retriever = retriever
        self.llm = llm
        self.history = []  # Store query history

    def query(
        self,
        question: str,
        top_k: int = 5,
        min_score: float = 0.2,
        stream: bool = False,
        summarize: bool = False,
    ) -> Dict[str, Any]:
        # Retrieve relevant documents
        results = self.retriever.retrieve(
            question, top_k=top_k, score_threshold=min_score
        )
        if not results:
            answer = "No relevant context found."
            sources = []
            context = ""
        else:
            context = "\n\n".join([doc["content"] for doc in results])
            sources = [
                {
                    "source": doc["metadata"].get(
                        "source_file", doc["metadata"].get("source", "unknown")
                    ),
                    "page": doc["metadata"].get("page", "unknown"),
                    "score": doc["similarity_score"],
                    "preview": doc["content"][:120] + "...",
                }
                for doc in results
            ]
            # Streaming answer simulation
            prompt = f"""Use the following context to answer the question concisely.\nContext:\n{context}\n\nQuestion: {question}\n\nAnswer:"""
            if stream:
                print("Streaming answer:")
                for i in range(0, len(prompt), 80):
                    print(prompt[i : i + 80], end="", flush=True)
                    time.sleep(0.05)
                print()
            response = self.llm.invoke(
                [prompt.format(context=context, question=question)]
            )
            answer = response.content

        # Add citations to answer
        citations = [
            f"[{i + 1}] {src['source']} (page {src['page']})"
            for i, src in enumerate(sources)
        ]
        answer_with_citations = (
            answer + "\n\nCitations:\n" + "\n".join(citations) if citations else answer
        )

        # Optionally summarize answer
        summary = None
        if summarize and answer:
            summary_prompt = f"Summarize the following answer in 2 sentences:\n{answer}"
            summary_resp = self.llm.invoke([summary_prompt])
            summary = summary_resp.content

        # Store query history
        self.history.append(
            {
                "question": question,
                "answer": answer,
                "sources": sources,
                "summary": summary,
            }
        )

        return {
            "question": question,
            "answer": answer_with_citations,
            "sources": sources,
            "summary": summary,
            "history": self.history,
        }


# Example usage:
adv_rag = AdvancedRAGPipeline(rag_retriever, llm)
result = adv_rag.query(
    "who is the author of this langchain pdf?",
    top_k=3,
    min_score=0.1,
    stream=True,
    summarize=True,
)
print("\nFinal Answer:", result["answer"])
print("Summary:", result["summary"])
print("History:", result["history"][-1])


Retrieving documents for query: 'who is the author of this langchain pdf?'
Top K: 3, Score threshold: 0.1


Batches: 100%|██████████| 1/1 [00:08<00:00,  8.03s/it]


Generated embeddings shape: (1, 384)
Retrieved 0 documents (after filtering)

Final Answer: No relevant context found.
Summary: There is no text to summarize. The original response indicates that no relevant context was found, providing no information to work with.
History: {'question': 'who is the author of this langchain pdf?', 'answer': 'No relevant context found.', 'sources': [], 'summary': 'There is no text to summarize. The original response indicates that no relevant context was found, providing no information to work with.'}
